# End-to-End Data Science/ML Project Template

**Author**: Arnav Sharma arnav.sharma2264@gmail.com
**Date**: 2025-08-29  
**Description**: A comprehensive template for building, evaluating, and deploying machine learning and deep learning models.

This notebook provides a structured framework for an end-to-end Data Science/ML project, covering data loading, preprocessing, exploratory data analysis (EDA), feature engineering, model training, AutoML, deep learning, evaluation, and deployment. The template is modular and adaptable for various datasets and ML tasks (classification, regression, etc.).

## Table of Contents
1. [Project Setup](#1-project-setup)
2. [Data Loading](#2-data-loading)
3. [Exploratory Data Analysis (EDA)](#3-exploratory-data-analysis-eda)
4. [Data Preprocessing](#4-data-preprocessing)
5. [Feature Engineering](#5-feature-engineering)
6. [Model Training](#6-model-training)
7. [AutoML](#7-automl)
8. [Deep Learning](#8-deep-learning)
9. [Model Evaluation](#9-model-evaluation)
10. [Hyperparameter Tuning](#10-hyperparameter-tuning)
11. [Model Interpretation](#11-model-interpretation)
12. [Model Deployment](#12-model-deployment)
13. [Conclusion and Next Steps](#13-conclusion-and-next-steps)

## 1. Project Setup
Install and import necessary libraries, set random seeds for reproducibility, and configure project settings.

In [1]:
# Install required packages (uncomment to install)
# !pip install -r requirements.txt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from xgboost import XGBClassifier, XGBRegressor
from flaml import AutoML
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import shap
from skopt import BayesSearchCV
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define project-specific variables
DATA_PATH = "path/to/your/dataset.csv"  # Update with your dataset path
TARGET_COLUMN = "target"  # Update with your target column name
TASK_TYPE = "classification"  # Options: "classification" or "regression"
MODEL_SAVE_PATH = "model.pkl"

/Users/aps/Desktop/ML-DL-Projects/env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Data Loading
Load the dataset and perform initial checks for data integrity.

In [ ]:
def load_data(file_path):
    """
    Load dataset from a given file path.
    
    Parameters:
    file_path (str): Path to the dataset (CSV, Excel, etc.)
    
    Returns:
    pandas.DataFrame: Loaded dataset
    """
    try:
        if file_path.endswith('.csv'):
            data = pd.read_csv(file_path)
        elif file_path.endswith('.xlsx'):
            data = pd.read_excel(file_path)
        else:
            raise ValueError("Unsupported file format")
        
        print("Data loaded successfully!")
        print(f"Shape: {data.shape}")
        print("\nFirst 5 rows:")
        print(data.head())
        print("\nData Info:")
        print(data.info())
        return data
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

# Load the dataset
df = load_data(DATA_PATH)
if df is None:
    raise SystemExit("Data loading failed. Exiting.")

## 3. Exploratory Data Analysis (EDA)
Understand the dataset through visualizations and statistical summaries.

In [ ]:
def perform_eda(df, target_column):
    """
    Perform exploratory data analysis on the dataset.
    
    Parameters:
    df (pandas.DataFrame): Input dataset
    target_column (str): Name of the target column
    """
    # Summary statistics
    print("\nSummary Statistics:")
    print(df.describe(include='all'))
    
    # Missing values
    print("\nMissing Values:")
    print(df.isnull().sum())
    
    # Distribution of target variable
    plt.figure(figsize=(8, 6))
    if TASK_TYPE == "classification":
        sns.countplot(x=target_column, data=df)
        plt.title(f"Distribution of {target_column}")
    else:
        sns.histplot(df[target_column], kde=True)
        plt.title(f"Distribution of {target_column} (Regression)")
    plt.show()
    
    # Correlation matrix (for numeric columns)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    plt.figure(figsize=(10, 8))
    sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', fmt='.2f')
    plt.title("Correlation Matrix")
    plt.show()
    
    # Pairplot for numeric features (optional, for smaller datasets)
    # sns.pairplot(df[numeric_cols])
    # plt.show()

# Perform EDA
perform_eda(df, TARGET_COLUMN)

## 4. Data Preprocessing
Clean the data and prepare it for modeling.

In [ ]:
def preprocess_data(df, target_column):
    """
    Preprocess the dataset (handle missing values, encode categorical variables, etc.).
    
    Parameters:
    df (pandas.DataFrame): Input dataset
    target_column (str): Name of the target column
    
    Returns:
    pandas.DataFrame: Preprocessed dataset
    """
    # Separate features and target
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Handle missing values
    numeric_cols = X.select_dtypes(include=[np.number]).columns
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns
    
    # Impute numeric columns with median
    for col in numeric_cols:
        X[col].fillna(X[col].median(), inplace=True)
    
    # Impute categorical columns with mode
    for col in categorical_cols:
        X[col].fillna(X[col].mode()[0], inplace=True)
    
    # Encode categorical variables
    le = LabelEncoder()
    for col in categorical_cols:
        X[col] = le.fit_transform(X[col])
    
    # Encode target variable (for classification)
    if TASK_TYPE == "classification":
        y = le.fit_transform(y)
    
    print("Data preprocessing completed!")
    return X, y

# Preprocess data
X, y = preprocess_data(df, TARGET_COLUMN)

## 5. Feature Engineering
Create new features and scale the data.

In [ ]:
def feature_engineering(X):
    """
    Perform feature engineering (e.g., creating new features, scaling).
    
    Parameters:
    X (pandas.DataFrame): Feature dataset
    
    Returns:
    pandas.DataFrame: Engineered feature dataset
    """
    # Example: Create interaction features (customize as needed)
    if 'feature1' in X.columns and 'feature2' in X.columns:
        X['feature1_feature2_interaction'] = X['feature1'] * X['feature2']
    
    # Scale numeric features
    scaler = StandardScaler()
    numeric_cols = X.select_dtypes(include=[np.number]).columns
    X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
    
    print("Feature engineering completed!")
    return X, scaler

# Perform feature engineering
X, scaler = feature_engineering(X)

## 6. Model Training
Split data and train multiple models.

In [ ]:
def train_models(X, y):
    """
    Train multiple machine learning models, AutoML, and a deep learning model.
    
    Parameters:
    X (pandas.DataFrame): Features
    y (numpy.array): Target
    
    Returns:
    dict: Trained models (including AutoML and Deep Learning)
    """
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Initialize models based on task type
    models = {}
    if TASK_TYPE == "classification":
        models['Logistic Regression'] = LogisticRegression(random_state=42)
        models['Random Forest'] = RandomForestClassifier(random_state=42)
        models['XGBoost'] = XGBClassifier(random_state=42)
    else:
        models['Linear Regression'] = LinearRegression()
        models['Random Forest'] = RandomForestRegressor(random_state=42)
        models['XGBoost'] = XGBRegressor(random_state=42)
    
    # Train manual models
    for name, model in models.items():
        model.fit(X_train, y_train)
        print(f"{name} trained successfully!")
    
    # Run AutoML
    automl_model = run_automl(X_train, y_train, TASK_TYPE, time_budget=60)
    models['AutoML'] = automl_model
    
    # Train deep learning model
    dl_model = train_deep_learning_model(X_train, y_train, TASK_TYPE)
    models['Deep Learning'] = dl_model
    
    return models, X_train, X_test, y_train, y_test

# Train models
models, X_train, X_test, y_train, y_test = train_models(X, y)

## 7. AutoML
Use an AutoML framework (FLAML) to automatically select and tune the best model for the task.

In [ ]:
from flaml import AutoML

def run_automl(X_train, y_train, task_type, time_budget=60):
    """
    Run AutoML using FLAML to automatically select and tune the best model.
    
    Parameters:
    X_train (pandas.DataFrame): Training features
    y_train (numpy.array): Training target
    task_type (str): 'classification' or 'regression'
    time_budget (int): Time budget in seconds for AutoML (default: 60)
    
    Returns:
    model: Best AutoML model
    """
    automl = AutoML()
    automl_settings = {
        "time_budget": time_budget,  # Time budget in seconds
        "metric": "accuracy" if task_type == "classification" else "rmse",
        "task": task_type,
        "log_file_name": "automl.log",
        "n_jobs": -1,
        "random_state": 42
    }
    
    automl.fit(X_train=X_train, y_train=y_train, **automl_settings)
    print(f"Best AutoML model: {automl.best_estimator}")
    print(f"Best configuration: {automl.best_config}")
    
    return automl

# Run AutoML
automl_model = run_automl(X_train, y_train, TASK_TYPE, time_budget=60)

## 8. Deep Learning
Train a deep learning model using TensorFlow/Keras for the task.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

def build_deep_learning_model(input_dim, task_type):
    """
    Build a simple feedforward neural network for classification or regression.
    
    Parameters:
    input_dim (int): Number of input features
    task_type (str): 'classification' or 'regression'
    
    Returns:
    keras.Model: Compiled Keras model
    """
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    
    # Output layer
    if task_type == "classification":
        model.add(Dense(1, activation='sigmoid'))  # Binary classification
        loss = 'binary_crossentropy'
        metrics = ['accuracy']
    else:
        model.add(Dense(1))  # Regression
        loss = 'mse'
        metrics = ['mse']
    
    model.compile(optimizer='adam', loss=loss, metrics=metrics)
    return model

def train_deep_learning_model(X_train, y_train, task_type):
    """
    Train the deep learning model with early stopping.
    
    Parameters:
    X_train (pandas.DataFrame): Training features
    y_train (numpy.array): Training target
    task_type (str): 'classification' or 'regression'
    
    Returns:
    keras.Model: Trained model
    """
    model = build_deep_learning_model(X_train.shape[1], task_type)
    
    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=100,
        batch_size=32,
        callbacks=[early_stopping],
        verbose=1
    )
    
    # Plot training history
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
    return model

# Train deep learning model
dl_model = train_deep_learning_model(X_train, y_train, TASK_TYPE)
models['Deep Learning'] = dl_model

## 9. Model Evaluation
Evaluate models using appropriate metrics.

In [ ]:
def evaluate_models(models, X_test, y_test):
    """
    Evaluate trained models using appropriate metrics.
    
    Parameters:
    models (dict): Dictionary of trained models
    X_test (pandas.DataFrame): Test features
    y_test (numpy.array): Test target
    """
    results = {}
    
    for name, model in models.items():
        if name == 'Deep Learning':
            y_pred = model.predict(X_test)
            y_pred = (y_pred > 0.5).astype(int).flatten() if TASK_TYPE == "classification" else y_pred.flatten()
        else:
            y_pred = model.predict(X_test)
        
        if TASK_TYPE == "classification":
            results[name] = {
                'Accuracy': accuracy_score(y_test, y_pred),
                'Precision': precision_score(y_test, y_pred, average='weighted'),
                'Recall': recall_score(y_test, y_pred, average='weighted'),
                'F1 Score': f1_score(y_test, y_pred, average='weighted')
            }
        else:
            results[name] = {
                'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
                'R2 Score': r2_score(y_test, y_pred)
            }
    
    # Display results
    results_df = pd.DataFrame(results).T
    print("\nModel Evaluation Results:")
    print(results_df)
    
    return results_df

# Evaluate models
results_df = evaluate_models(models, X_test, y_test)

## 10. Hyperparameter Tuning
Optimize the best-performing model using Bayesian optimization.

In [ ]:
def tune_best_model(X_train, y_train, best_model_name, models):
    """
    Perform hyperparameter tuning on the best model (excluding AutoML and Deep Learning).
    
    Parameters:
    X_train (pandas.DataFrame): Training features
    y_train (numpy.array): Training target
    best_model_name (str): Name of the best model
    models (dict): Dictionary of trained models
    
    Returns:
    model: Tuned model
    """
    # Define hyperparameter search space (customize as needed)
    param_space = {
        'Random Forest': {
            'n_estimators': (10, 200),
            'max_depth': (3, 20),
            'min_samples_split': (2, 10)
        },
        'XGBoost': {
            'n_estimators': (10, 200),
            'max_depth': (3, 20),
            'learning_rate': (0.01, 0.3, 'log-uniform')
        }
    }
    
    if best_model_name in param_space:
        bayes_cv = BayesSearchCV(
            estimator=models[best_model_name],
            search_spaces=param_space[best_model_name],
            n_iter=20,
            cv=5,
            n_jobs=-1,
            random_state=42
        )
        bayes_cv.fit(X_train, y_train)
        print(f"Best parameters for {best_model_name}: {bayes_cv.best_params_}")
        return bayes_cv.best_estimator_
    elif best_model_name in ['AutoML', 'Deep Learning']:
        print(f"{best_model_name} model is already tuned or does not support BayesSearchCV. Skipping additional tuning.")
        return models[best_model_name]
    else:
        print(f"No tuning defined for {best_model_name}")
        return models[best_model_name]

# Select the best model (based on evaluation metrics)
best_model_name = results_df.idxmax()['Accuracy' if TASK_TYPE == "classification" else 'R2 Score']
tuned_model = tune_best_model(X_train, y_train, best_model_name, models)

## 11. Model Interpretation
Interpret the model using SHAP values.

In [ ]:
def interpret_model(model, X_train, model_name):
    """
    Interpret the model using SHAP values.
    
    Parameters:
    model: Trained model
    X_train (pandas.DataFrame): Training features
    model_name (str): Name of the model
    """
    try:
        if model_name in ['Random Forest', 'XGBoost']:
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X_train)
        elif model_name == 'AutoML':
            # Check if the AutoML model is tree-based
            if 'rf' in model.best_estimator.lower() or 'xgboost' in model.best_estimator.lower():
                explainer = shap.TreeExplainer(model.model)
                shap_values = explainer.shap_values(X_train)
            else:
                explainer = shap.KernelExplainer(model.predict, X_train)
                shap_values = explainer.shap_values(X_train)
        elif model_name == 'Deep Learning':
            explainer = shap.DeepExplainer(model, X_train)
            shap_values = explainer.shap_values(X_train.to_numpy())
        else:
            explainer = shap.KernelExplainer(model.predict, X_train)
            shap_values = explainer.shap_values(X_train)
        
        # Summary plot
        shap.summary_plot(shap_values, X_train, plot_type="bar")
        plt.title(f"Feature Importance (SHAP) - {model_name}")
        plt.show()
    except Exception as e:
        print(f"SHAP interpretation failed for {model_name}: {e}")

# Interpret the best model
interpret_model(tuned_model, X_train, best_model_name)

## 12. Model Deployment
Save the model and create a prediction function for deployment.

In [ ]:
def save_model(model, scaler, path, model_name):
    """
    Save the trained model and scaler.
    
    Parameters:
    model: Trained model
    scaler: Fitted scaler
    path (str): Path to save the model
    model_name (str): Name of the model
    """
    if model_name == 'Deep Learning':
        model.save(path.replace('.pkl', '.h5'))  # Save Keras model in HDF5 format
        joblib.dump({'scaler': scaler}, path.replace('.pkl', '_scaler.pkl'))
        print(f"Deep Learning model saved to {path.replace('.pkl', '.h5')}")
        print(f"Scaler saved to {path.replace('.pkl', '_scaler.pkl')}")
    else:
        joblib.dump({'model': model, 'scaler': scaler}, path)
        print(f"Model and scaler saved to {path}")

def predict_new_data(model, scaler, new_data, model_name):
    """
    Make predictions on new data.
    
    Parameters:
    model: Trained model
    scaler: Fitted scaler
    new_data (pandas.DataFrame): New data for prediction
    model_name (str): Name of the model
    
    Returns:
    numpy.array: Predictions
    """
    # Preprocess new data
    numeric_cols = new_data.select_dtypes(include=[np.number]).columns
    new_data[numeric_cols] = scaler.transform(new_data[numeric_cols])
    
    # Make predictions
    if model_name == 'Deep Learning':
        predictions = model.predict(new_data).flatten()
        if TASK_TYPE == "classification":
            predictions = (predictions > 0.5).astype(int)
    else:
        predictions = model.predict(new_data)
    return predictions

# Save the model
save_model(tuned_model, scaler, MODEL_SAVE_PATH, best_model_name)

# Example: Predict on new data (replace with actual new data)
# new_data = pd.DataFrame(...)  # Load or create new data
# predictions = predict_new_data(tuned_model, scaler, new_data, best_model_name)
# print("Predictions on new data:", predictions)

## 13. Conclusion and Next Steps
- **Summary**: Summarize the findings, best model performance (including AutoML and Deep Learning results), and key insights from EDA and model interpretation.
- **Next Steps**: Consider additional feature engineering, trying other AutoML frameworks, experimenting with different neural network architectures, or deploying the model in a production environment.
- **Monitoring**: Plan for model monitoring and retraining to handle data drift.

```python
print("End-to-End ML and Deep Learning Project Completed!")
```

In [ ]:
print("End-to-End ML and Deep Learning Project Completed!")